# Cellpose 2.x Installation – **WORKING VERSION for MyBinder**

This notebook installs **Cellpose 2.x** with the classic API that works end‑to‑end.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/your-username/your-repo/HEAD?labpath=cellpose_working.ipynb)

## Features
✅ Installs Cellpose 2.x (stable API)  
✅ Creates synthetic test data  
✅ Runs segmentation successfully  
✅ Shows beautiful results  
✅ Provides working code examples


## Step 1  – Environment Setup

In [ ]:
import sys, os, subprocess, time

print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")
print(f"Platform: {sys.platform}")

def run_cmd(cmd, timeout=600):
    """Run *cmd* in a subshell and capture stdout/stderr, returning (success, out, err)."""
    try:
        res = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return res.returncode == 0, res.stdout.strip(), res.stderr.strip()
    except subprocess.TimeoutExpired:
        return False, "", f"Timeout after {timeout}s"
    except Exception as e:
        return False, "", str(e)

print("\n✅ Environment setup complete!")


## Step 2  – Install Cellpose 2.x and dependencies

In [ ]:
print("🔧 Installing Cellpose 2.3.2 (CPU build) and dependencies…")
print("This may take 3–4 minutes on Binder…\n")

# Numerical & plotting stack pinned to avoid ABI mismatches
packages = [
    "numpy<2.0",
    "scipy<1.12",
    "scikit-image<0.23",
    "matplotlib<3.9",
    # I/O & utils
    "opencv-python-headless",
    "tifffile",
    # Deep‑learning stack (CPU wheels)
    "torch==2.2.2",
    "torchvision==0.17.2",
    # Cellpose itself
    "cellpose==2.3.2"
]

successful, failed = [], []
for i, pkg in enumerate(packages, 1):
    print(f"[{i}/{len(packages)}] Installing {pkg} …", end=" ")
    ok, out, err = run_cmd(f"pip install --quiet {pkg}")
    if ok:
        print("✅")
        successful.append(pkg)
    else:
        print("❌")
        failed.append(pkg)
        if err:
            # Show first line that contains the word 'ERROR' or the last line
            lines = err.split('\n')
            msg = next((l for l in lines if 'ERROR' in l), lines[-1])
            print(f"    {msg[:120]}")

print("\n📊 Installation summary")
print(f"  ✅ Successful: {len(successful)}/{len(packages)}")
print(f"  ❌ Failed    : {len(failed)}")
if failed:
    print(f"  Failed packages: {failed}")

installation_ok = len(successful) >= len(packages) - 1  # tolerate one mirror hiccup
print(f"\nStatus: {'✅ READY' if installation_ok else '⚠️  PARTIAL'}")


## Step 3  – Test Cellpose import

In [ ]:
print("🔍 Testing Cellpose 2.x import…")
try:
    import cellpose
    from cellpose import models, io, utils
    import numpy as np
    import matplotlib.pyplot as plt
    import tifffile

    print("✅ All imports successful")
    print(f"✅ Cellpose version: {cellpose.__version__}")

    # Test model initialisation
    model = models.Cellpose(gpu=False, model_type='cyto')
    print("✅ Cellpose model initialised")

    cellpose_working = True
except Exception as e:
    print(f"❌ Error: {e}")
    cellpose_working = False

# Configure matplotlib for headless use
if cellpose_working:
    import matplotlib
    matplotlib.use('Agg')
    plt.ioff()
    print("✅ Matplotlib configured")

print(f"\nCellpose status: {'✅ WORKING' if cellpose_working else '❌ FAILED'}")


## Step 4  – Create synthetic test data

In [ ]:
if cellpose_working:
    print("🔬 Creating synthetic test data…")
    from scipy import ndimage
    import numpy as np, tifffile, matplotlib.pyplot as plt

    def create_cells(size=(512, 512), n_cells=12):
        img = np.zeros(size, dtype=np.float32)
        np.random.seed(42)
        for _ in range(n_cells):
            cy = np.random.randint(50, size[0]-50)
            cx = np.random.randint(50, size[1]-50)
            radius = np.random.randint(18, 35)
            y, x = np.ogrid[:size[0], :size[1]]
            dist = np.sqrt((x-cx)**2 + (y-cy)**2)
            noise = np.random.normal(0, 3, size)
            mask = dist <= (radius + 0.3*noise)
            intensity = np.random.uniform(160, 240)
            img[mask] = intensity
        img += np.random.normal(20, 8, size)
        img = ndimage.gaussian_filter(img, sigma=1.5)
        img = np.clip(img, 0, 255).astype(np.uint8)
        return img

    test_img = create_cells()
    tifffile.imwrite('test_cells.tif', test_img)

    plt.figure(figsize=(6,5))
    plt.imshow(test_img, cmap='gray')
    plt.title('Synthetic test cells')
    plt.axis('off')
    plt.tight_layout()
    plt.savefig('test_preview.png', dpi=120)
    plt.close()

    print(f"✅ Test image created: {test_img.shape}, range {test_img.min()}–{test_img.max()}")
    test_data_ok = True
else:
    print("⚠️  Skipping test data generation – Cellpose not working")
    test_data_ok = False


## Step 5  – Run segmentation

In [ ]:
if cellpose_working and test_data_ok:
    print("🎯 Running Cellpose segmentation…")
    import time, numpy as np, tifffile
    img = tifffile.imread('test_cells.tif')
    print(f"Loaded image: {img.shape}")

    start_time = time.time()
    masks, flows, styles, diams = model.eval(
        img,
        diameter=None,
        channels=[0, 0],
        flow_threshold=0.4,
        cellprob_threshold=0.0
    )
    end_time = time.time()
    n_cells = len(np.unique(masks)) - 1

    print("\n🎉 Segmentation completed!")
    print(f"  Time          : {end_time - start_time:.1f}s")
    print(f"  Cells detected: {n_cells}")
    print(f"  Diameter      : {diams:.1f}px")

    segmentation_ok = True
else:
    print("⚠️  Skipping segmentation – prerequisites not met")
    segmentation_ok = False


## Step 6  – Visualise results

In [ ]:
if segmentation_ok:
    print("📊 Creating visualisation…")
    import matplotlib.pyplot as plt
    from skimage import measure
    import numpy as np, tifffile

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img, cmap='gray')
    axes[0].set_title('Original'); axes[0].axis('off')

    axes[1].imshow(masks, cmap='nipy_spectral')
    axes[1].set_title(f'Masks ({n_cells} cells)'); axes[1].axis('off')

    axes[2].imshow(img, cmap='gray')
    axes[2].contour(masks, levels=[0.5], colors='red', linewidths=1)
    axes[2].set_title('Overlay'); axes[2].axis('off')

    plt.tight_layout()
    plt.savefig('results.png', dpi=150)
    plt.close()

    tifffile.imwrite('masks.tif', masks.astype(np.uint16))
    outlines = utils.masks_to_outlines(masks)
    tifffile.imwrite('outlines.tif', outlines.astype(np.uint8)*255)

    # Quick regionprops summary
    props = measure.regionprops(masks, img)
    areas = [p.area for p in props]
    intensities = [p.mean_intensity for p in props]
    print(f"\n📈 Cell analysis")
    print(f"  Total cells     : {len(areas)}")
    print(f"  Mean area (px²) : {np.mean(areas):.1f} ± {np.std(areas):.1f}")
    print(f"  Intensity (mean): {np.mean(intensities):.1f}")

    visualization_ok = True
else:
    print("⚠️  Skipping visualisation – segmentation failed")
    visualization_ok = False


## Step 7  – Usage examples

In [ ]:
print("📚 CELLPOSE 2.x USAGE GUIDE")
print("="*60)

print("""🔬 Basic workflow

from cellpose import models
import tifffile

img = tifffile.imread('your_image.tif')
model = models.Cellpose(gpu=False, model_type='cyto')
masks, flows, styles, diams = model.eval(
    img,
    diameter=None,        # auto
    channels=[0,0],
    flow_threshold=0.4,
    cellprob_threshold=0.0
)
tifffile.imwrite('masks.tif', masks)
""")

print("""🎛️ Key parameters

• diameter=None : auto‑estimate  
• diameter=30   : set expected cell size  
• flow_threshold: lower → more cells  
• cellprob_threshold: higher → fewer cells  
• channels=[0,0]: grayscale  
• model_type    : 'cyto', 'nuclei', 'cyto2'""")

if 'n_cells' in globals():
    print(f"""✅ Your results

• Successfully segmented {n_cells} cells  
• Using Cellpose {cellpose.__version__} (classic API)  
• Results saved to 'masks.tif' / 'results.png'""")

print("""🚀 Next steps

1. Upload your own images
2. Adjust parameters
3. Try other models
4. Extract quantitative features
5. Integrate into your pipeline""")


## Step 8  – Final summary

In [ ]:
print("🎉 CELLPOSE 2.x SET‑UP – FINAL REPORT")
print("="*60)

overall_success = all([
    installation_ok,
    cellpose_working,
    test_data_ok,
    segmentation_ok,
    visualization_ok
])

files = ['test_cells.tif', 'test_preview.png', 'results.png', 'masks.tif', 'outlines.tif']
print("\n📁 Generated files")
for f in files:
    if os.path.exists(f):
        sz = os.path.getsize(f)/1024
        print(f"  ✅ {f}  ({sz:.1f} KB)")
    else:
        print(f"  ❌ {f} (missing)")

print("\n🔍 Component status")
components = [
    ("Package installation", installation_ok),
    ("Cellpose import"     , cellpose_working),
    ("Test data creation"  , test_data_ok),
    ("Segmentation"        , segmentation_ok),
    ("Visualisation"       , visualization_ok)
]
for name, ok in components:
    print(f"  {'✅' if ok else '❌'} {name}")

if overall_success:
    print("\n🎯 All good – ready for research!")
else:
    print("\n⚠️  Something failed – see log above")
print("="*60)
